<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/11-Flask/B-Create_API_call_and_Connecting_to_MySQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating of a Flask application showing Citibike stations


In this segment we will create a basic app, where we will connect to the Citibike database, and display the list of stations. 



In [ ]:
!pip install -U PyMySQL sqlalchemy flask flask-ngrok pyngrok

In [ ]:
ngrok_authtoken = '2EYf3qVk9mi739HjPwSNZXWAtfy_4jF9NAhGqVVVJmm4YehPW'

## A Refresher 

Let's remember first how to get data from the database.

In [ ]:
# This code creates a connection to the database
from sqlalchemy import create_engine

conn_string = 'mysql+pymysql://{user}:{password}@{host}/{db}?charset={encoding}'.format(
    host = 'db.ipeirotis.org', 
    user = 'student',
    db = 'citibike_fall2017',
    password = 'dwdstudent2015',
    encoding = 'utf8mb4')

engine = create_engine(conn_string)

In [ ]:
sql = "SELECT DISTINCT id, name, capacity, lat, lon  FROM status_fall2017 LIMIT 5"

with engine.connect() as connection:
  stations = connection.execute(sql).mappings().all()

for s in stations:
    print(s)
    print("-----")

## Creating an API endpoint

In [ ]:
import os
import threading

from flask import Flask
from pyngrok import ngrok

os.environ["FLASK_ENV"] = "development"

app = Flask(__name__)
port = 5000

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel '{public_url}' -> 'http://127.0.0.1:{port}'")

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL


# Define Flask routes
@app.route("/")
def index():
    return "Hello from Colab!"

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

In [ ]:
from flask import Flask, render_template, jsonify
from flask_ngrok import run_with_ngrok
from sqlalchemy import create_engine
import pandas as pd
import base64
from io import BytesIO
import matplotlib.pyplot as plt

app = Flask(__name__)
run_with_ngrok(app)
 
# Setup a connection to the database
conn_string = 'mysql+pymysql://{user}:{password}@{host}/{db}?charset={encoding}'.format(
    host = 'db.ipeirotis.org', 
    user = 'student',
    db = 'citibike_fall2017',
    password = 'dwdstudent2015',
    encoding = 'utf8mb4')
engine = create_engine(conn_string)

#### Now let's start our server:

A few things to notice.

First, notice the `@app.route('/citibike_api',  methods=['GET'])` command. This part specifies that our API endpoint will be accessible under the `http://our_web_server_address/citibike_api`.


Then, we define the function `def citibike_stations():` that will create the response of that API call. What the function returns is what the API call will return back.

Notice that insider the `citibike_stations` function, we connect to the database, and issue an SQL query to the database.

Finally, we get back the results of the query, we put the results in a Python dictionary, and we use the `jsonify` function to convert our dictionary to JSON and return it as the API result.


In [ ]:
@app.route('/citibike_api',  methods=['GET'])
def citibike_stations():


  # Connect to the database, execute the query, and get back the results
  sql = "SELECT DISTINCT id, name, capacity, lat, lon  FROM status_fall2017" 
  with engine.connect() as connection:  
    rows = connection.execute(sql).mappings().all()
    
  # Create the response. We will put the retrieved data as a list of
  # dictionaries, under the key "stations".
  results = {"stations": []}
  for r in rows:
    results["stations"].append(dict(r))

  # We JSON-ify our dictionary and return it as the API response
  return jsonify(results)
 


In [ ]:
@app.route('/station_map',  methods=['GET'])
def station_map():


  # Connect to the database, execute the query, and get back the results
  sql = "SELECT DISTINCT id, name, capacity, lat, lon  FROM status_fall2017" 
  with engine.connect() as connection:  
    df = pd.read_sql(sql, con=connection)

  fig,ax = plt.subplots()
    
  df.plot(kind='scatter', x='lat', y='lon', ax=ax)

  buf = BytesIO()
  fig.savefig(buf, format="png")
  # Embed the result in the html output.
  data = base64.b64encode(buf.getbuffer()).decode("ascii")

  # Create the response. We will put the retrieved data as a list of
  # dictionaries, under the key "stations".
  results = {"image": data}

  # We JSON-ify our dictionary and return it as the API response
  return jsonify(results)

  # return f"<img src='data:image/png;base64,{data}'/>"
 


In [ ]:
app.run()